<a href="https://colab.research.google.com/github/111221007/FSAI/blob/main/onnx_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics opencv-python-headless


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 128.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 108.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [16]:
import cv2
import torch
from ultralytics import YOLO
from pathlib import Path
import datetime
import time
import os

In [17]:
from email_sender import send_accident_email


In [18]:
MODEL_PATH = Path("/content/model/pruned_model.onnx")
VIDEO_PATH = Path("/content/data/input/1.mp4")
SAVE_DIR = Path("/content/data/output")
CONFIDENCE_THRESHOLD = 0.5
CONSECUTIVE_FALL_FRAMES = 3
MAX_FALL_FRAMES = 5


In [22]:
# ======== FALL DETECTION LOGIC ========
def detect_falls(video_path, save_dir):
    save_dir.mkdir(parents=True, exist_ok=True)
    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        print("❌ Cannot open selected video.")
        return

    # Remove .to() as it is not supported for ONNX models
    model = YOLO(str(MODEL_PATH))

    streak = 0
    frame_count = 0
    fall_frames = 0
    email_sent = False
    fall_detected_time = None
    fall_detected_frame_capture_time = None

    first_inference_latency_ms = None
    total_inference_latency_ms = 0
    inference_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1
        frame_capture_time = time.time()

        # Inference latency measurement
        start_infer = time.time()
        # Pass device argument directly to predict
        results = model.predict(frame, imgsz=640, conf=CONFIDENCE_THRESHOLD, verbose=False, device='cuda' if torch.cuda.is_available() else 'cpu')[0]
        end_infer = time.time()
        current_inference_latency_ms = (end_infer - start_infer) * 1000

        if first_inference_latency_ms is None:
            first_inference_latency_ms = current_inference_latency_ms

        total_inference_latency_ms += current_inference_latency_ms
        inference_count += 1


        boxes = results.boxes
        fall_detected = any(
            model.names[int(box.cls[0])] == "down" and float(box.conf[0]) > CONFIDENCE_THRESHOLD for box in boxes
        )

        if fall_detected and fall_detected_time is None:
            fall_detected_time = time.time()
            fall_detected_frame_capture_time = frame_capture_time  # For end-to-end latency

        if fall_detected:
            streak += 1
        else:
            streak = 0

        if streak >= CONSECUTIVE_FALL_FRAMES and fall_frames < MAX_FALL_FRAMES:
            fall_frames += 1
            timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
            save_path = save_dir / f"fall_frame_{fall_frames:04d}.jpg"
            cv2.imwrite(str(save_path), frame)
            print(f"Fall detected! Saved: {save_path}")

            if not email_sent:
                # Send email (blocking, as in your code)
                send_accident_email(save_dir)
                alert_confirmed_time = time.time()
                fall_to_alert_latency_ms = (alert_confirmed_time - fall_detected_time) * 1000
                end_to_end_latency_ms = (alert_confirmed_time - fall_detected_frame_capture_time) * 1000
                print(f"Latency from fall detection to alert confirmation: {fall_to_alert_latency_ms:.2f} ms")
                print(f"End-to-end latency (frame capture to alert confirmation): {end_to_end_latency_ms:.2f} ms")
                email_sent = True

            streak = 0

    cap.release()
    print("Detection complete.")
    if inference_count > 0:
        average_inference_latency_ms = total_inference_latency_ms / inference_count
        print(f"First inference latency: {first_inference_latency_ms:.2f} ms")
        print(f"Average inference latency: {average_inference_latency_ms:.2f} ms")

In [23]:
# ======== RUN FALL DETECTION ========
detect_falls(VIDEO_PATH, SAVE_DIR)

WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
Loading /content/model/pruned_model.onnx for ONNX Runtime inference...
Using ONNX Runtime CUDAExecutionProvider
Fall detected! Saved: /content/data/output/fall_frame_0001.jpg
✅ Fall alert email sent successfully!
Latency from fall detection to alert confirmation: 3629.84 ms
End-to-end latency (frame capture to alert confirmation): 3644.21 ms
Fall detected! Saved: /content/data/output/fall_frame_0002.jpg
Fall detected! Saved: /content/data/output/fall_frame_0003.jpg
Fall detected! Saved: /content/data/output/fall_frame_0004.jpg
Fall detected! Saved: /content/data/output/fall_frame_0005.jpg
Detection complete.
First inference latency: 243.79 ms
Average inference latency: 16.66 ms
